# Tutorial 2b: Running a generic single-point case

This tutorial is an introduction to running single-point simulations of the Community Terrestrial Systems Model (CTSM).  It will guide you through setting up the required driver data (i.e. surface and atmosphere data) as well as setting up and submitting a single-point CTSM case.

In the previous tutorial, `Day1_GlobalCase`, we set up and ran a global CTSM case. Many of the steps required to run a single-point case are similar, with some changes and additional required steps which we will cover here.
<br><br>

### Questions about this tutorial? 
- Please post them on the [CTSM forum in the CESM Bulletin Board](https://bb.cgd.ucar.edu/cesm/forums/ctsm-clm-mosart-rtm.134/). Note that this resource will require you to register and log in so that you can be notified of responses to your inquiries. 
- You can also file issues on the [NCAR CTSM-Tutorial GitHub repository](https://github.com/NCAR/CTSM-Tutorial-2022).
***

## In this tutorial

The tutorial has several components. Below you will find steps to: 
1. Generate subset surface and atmosphere data files at a single latitude and longitude point.
2. Set up and submit a single-point case.

<div class="alert alert-block alert-warning">
<b>Note:</b> This tutorial assumes you have completed the previous tutorials! If you haven't completed the Day 0 and Day1 tutorials, go back and do these first.
</div>

***

<h1> 1. Subset global surface and atmosphere files </h1>

CTSM uses a surface data file to read in important grid cell-level information like vegetation, crop, and glacier grid cell fractions, the fractional cover of each plant functional type (PFT), and soil characteristics.

A global surface data file is located and read by default for global CTSM cases, depending on the chosen _component set_ and _resolution_. To run CTSM at a single point, we will need to supply a surface data file at a specified latitude and longitude.


<div class="alert alert-block alert-info">
Remember from Day 1 that a <b>component set</b>, or coloquially a "compset", specifies a configuration for your case, including the component models, time period of simulation, and physics options. The <b>resolution</b> defines the model resolution or grid.
</div>


Similarly, when running in _DATM mode_, that is when using a "data atmosphere model", with climate data (e.g. temperature, precipitation, solar radiation, etc.) driven by an input file, CTSM needs DATM files. We can also provide subset global DATM for single-point runs.

<div class="alert alert-block alert-info">
<b>Tip:</b> It is not strictly required to provide CTSM/CIME with subset DATM data in order to run a single-point case, as CTSM can just use the global files. However, your runs will be much faster if you use subset climate data.
</div>

<h2> 1.1 Use <em>subset_data</em> to subset surface and DATM files</h2>

We have created a python script, *subset_data*, which will subset default global surface and DATM files at a user-specified latitude and longitude.

This script is located in the CTSM source code, in the `tools/site_and_regional` folder.

Navigate here now:

In [ ]:
cd ~/CTSM/tools/site_and_regional
ls

<div>
<img src="../images/ls_subset_data.png" width="588" height="118" alt="Example output from an ls command in the correct folder"/>
</div>

In order to use this python script you must have some required python packages installed. On NCAR machines (like Cheyenne), you can load the NCAR python library, _ncar pylib_, by running `module load python` and then `ncar_pylib`. For this tutorial, we already have these python libraries available to you.

<div class="alert alert-block alert-info">
<b>Tip:</b> On NCAR or other machines, you can also use your own python environment if you want. Required third-party python packages are <i>scipy</i>, <i>xarray</i>, and <i>numpy</i>.
</div>

You can use the built-in print help to see what options are available for the subset data script:

In [ ]:
./subset_data --help

There are a lot of options, but for now we will just use a few:

`point` : this tells the script to subset data at a single point  
`--lat` : this tells the script which latitude to subset at (*must be between -90 and 90*)  
`--lon` : this tells the script which longitude to subset at (*can be between 0 and 360 or -180 and 180*)  
`--site` : optional, specifies a site name or tag  
`--create-surface` : tells the script to subset surface data  
`--create-datm` : tells the script to subset DATM data  
`--datm-syr` and `--datm-eyr`: starting and ending years for the DATM data to subset (*must be between 1901 and 2014*)  
`--create-user-mods` : tells the script to create a *user_mods* directory (see below)  
`--outdir` : specifies the directory to place subset data and user mods directory in

Call the script with these options. For this tutorial, we will use a latitude of 42.53562 and longitude of 287.82438 (i.e. Harvard Forest). Note that your latitude and longitude points do not have to be this precise for your own sites!

In [ ]:
./subset_data point --lat 42.53562 --lon 287.82438 --site my_point --create-surface --create-datm --datm-syr 2001 --datm-eyr 2010 --create-user-mods --outdir ~/my_subset_data

This may take a bit of time to subset all the climate data.

<h2> 1.2 Check on the subset files </h2>

Once the subsetting has successfully finished, let's navigate to the specified output directory to check on the files that we just created:

In [ ]:
cd ~/my_subset_data
ls

<div>
<img src="../images/ls_subset_folder.png" width="588" height="118" alt="Example output from an ls command in the subset data folder."/>
</div>

You should see a surface data file (e.g. *surfdata_0.9x1.25 ... .nc*) and two folders: **datmdata** and **user_mods**. 

* **datmdata** houses the subset DATM files
* **user_mods** is a directory created that houses several files we will use to set up our single-point case

Let's navigate into the **user_mods** directory to check them out:

In [ ]:
cd user_mods
ls

You should see three files: *shell_commands*, *user_nl_clm*, and *user_nl_datm_streams*.  

The *shell_commands* file contains *xmlchange* commands required to set up a single point case at the specified latitude and longitude.

Take a look at this file if you want:

In [ ]:
cat shell_commands

`./xmlchange CLM_USRDAT_DIR` - this tells CIME the location of an argument *CLM_USRDAT_DIR* which we can use to specify the main directory of subset data files  

`./xmlchange PTS_LON` and `./xmlchange PTS_LAT` - this tells CIME that we are running at a specified latitude and longitude  

`./xmlchange MPILIB` - this specifies a specific MPI library to use required for single-point runs.  
 
   
If you remember from the Day 1 tutorial, *user_nl_clm* is a Fortran namelist file used to set up different namelist options for CLM. Here, we are using it to specify the location of our subset surface data. Note the use of the variable `$CLM_USRDAT_DIR` set up in the *shell_commands* file.
  
    
Similarly, *user_nl_datm_streams* specifies the location and a few other options for our subset DATM data.


We will use this **user_mods** directory when we create our single-point case (see below).

<div class="alert alert-block alert-warning">
<b>Note:</b> If for whatever reason you end up moving the subset data directory (i.e. here <b>~/my_subset_data</b>), you will need to modify the xmlchange command that specifies the <i>CLM_USRDAT_DIR</i> to be the full path to the directory's new location. 
</div>

<h1> 2. Create a single-point CTSM case </h1>

Now that we have our subset data ready to go, we can set up our single-point case with CIME.

The steps required here are very similar to the global case that we set up in the Day 1 tutorial, with a few differences. Mainly, we are going to specify a `--user-mods-dir` in our `./create_newcase` command as the full path to the **user_mods** folder we just created with *subset_data*. We will also choose a different component set and resolution.

<h2> 2.1. Create the case </h2>

As in our Day 1 Tutorial, we will navigate into the CTSM **scripts** directory to run the `create_newcase` script. Today we are going to be running a CLM-BGC simulation.

In [ ]:
cd ~/CTSM/cime/scripts
./create_newcase --case ~/clm_tutorial_cases/I2000_CTSM_singlept --res CLM_USRDAT --compset I2000Clm50BgcCrop --run-unsupported --user-mods-dirs ~/my_subset_data/user_mods

This command should look fairly familiar to you, with some updated values and arguments.

`--res` - defines the model resolution, or grid:
* we are now using `CLM_USRDAT`, which should be used when we have user-specified domain (i.e. a subset surface data)

`--compset` - defines the component set for the case:
* `I2000Clm50BgcCrop` is an alias that describes using year 2000 initialization time, data-driven atmosphere (GSWP3v1 data), CLM 5.0 BGC with prognostic crop, along with some other component settings.

`--user-mods-dirs` - this is where we tell CIME where our **user mods** directory is.
* it should be the path to the directory that was created during our *subset_data* scripting
* the namelist files (i.e. *user_nl_clm* and *user_nl_datm_streams*) will be copied into the case directory, and the commands within *shell_commands* will be executed

<div class="alert alert-block alert-info">
<b>Tip:</b> Depending on your machine (e.g. this is required on Cheyenne), you may also have to provide a project id (<code>--project $PROJECT_ID</code>) which specifies accounting or directory permissions when on a batch system. By default the script uses the shell environment variable <code>$PROJECT</code>, which can be set in your bash profile

Remember that you can see script parameters and definitions using <code>./create_newcase --help</code>
</div>

<h2> 2.2. Set up the case and build the executable </h2>

As with our global case, we will change into our case directory, set up our case (`./case.setup`) and then build the executable (`./case.build`):

In [ ]:
cd ~/clm_tutorial_cases/I2000_CTSM_singlept
./case.setup
./case.build

<h2> 2.3. Customize the case </h2>

As in our global case, we will invoke a few XML commands to update some runtime values:

In [ ]:
./xmlchange STOP_OPTION=nyears
./xmlchange STOP_N=10
./xmlchange RUN_STARTDATE='2001-01-01'
./xmlchange DATM_YR_ALIGN=2001
./xmlchange DATM_YR_START=2001
./xmlchange DATM_YR_END=2010

<h3> Spinup </h3>

When running a model like CLM, the initial conditions (i.e. state variables like carbon and nitrogen pools and soil moisture) are very impactful on the overall results of the simulation. Often, we don't know the precise values of these initial conditions. To get around this issue, we can initialize the model with arbitrary values and then run the model with some cycle of atmospheric forcing for many years (e.g. 200) until the model attains an equilibrium state. Then, we can simulate the model response to some perturbation (e.g. changing climate, CO<sub>2</sub>, etc.). This is called _spinup_.

We have already run such a spinup simulation at Harvard Forest. We can see how the soil C pools evolved for this simulation over time.


<div class="alert alert-block alert-info">
This spinup simulation was conducted using "accelerated decomposition", or "AD" mode. We ran the model in this mode for 100 years cycling through atmospheric forcing for 1981 to 2000.

AD mode was invoked with the commands: <code>./xmlchange CLM_FORCE_COLDSTART=on</code> and <code>./xmlchange CLM_ACCELERATED_SPNIUP=on</code>.

We then ran a "post-AD" simulation (using the end of our "AD" simulation as the starting point) for another 100 years with </code>CLM_ACCELERATED_SPINUP=off</code> to allow the state variables to equilibrate with non-accelerated decomposition.
</div>


Now we just have to tell CIME to use the spinup simulation's end point as the starting point for our simulation. 

We do this via the <i>user_nl_clm</i> file:

In [ ]:
echo "finidat='path/to/file'" >> user_nl_clm

<div class="alert alert-block alert-info">
<b>Tip:</b> You can also do this using any text editing software (e.g. vi, emacs, etc.)
</div>

Let's check the file to make sure our command worked:

In [ ]:
cat user_nl_clm

<h2> 2.4. Submit the case </h2>

Finally, let's submit the case as we did in Day 1:

In [ ]:
./case.submit

You should see a confirmation that it successfully submitted.

### Congratulations! You've created and submtted a single-point CLM case!